# Classical Molecular Interaction Potentials tutorial using BioExcel Building Blocks (biobb)

***
This tutorial aims to illustrate the process of computing **classical molecular interaction potentials** from **protein structures**, step by step, using the **BioExcel Building Blocks library (biobb)**. The particular example used is the **XXXX** protein (PDB code [XXXX](https://www.rcsb.org/structure/XXXX)).   
***

## Settings

### Biobb modules used

 - [biobb_cmip](https://github.com/bioexcel/biobb_cmip): Tools to compute classical molecular interaction potentials from protein structures.
 - [biobb_structure_utils](https://github.com/bioexcel/biobb_structure_utils): Tools to modify or extract information from a PDB structure.
  
### Auxiliar libraries used

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [plotly](https://plotly.com/python/): Python Open Source Graphing Library. 


### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_cmip.git
cd biobb_wf_cmip
conda env create -f conda_env/environment.yml
conda activate biobb_wf_cmip
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-notebook biobb_wf_cmip/notebooks/biobb_wf_cmip.ipynb
  ``` 

***
## Pipeline steps
 1. [Input Parameters](#input)
 2. [PDB preparation (from PDB databank)](#preparePDB)
 3. [Molecular Interaction Potentials](#mips)
    1. [MIP+](#mip_pos) 
    2. [MIP-](#mip_neg) 
    3. [MIPn](#mip_neutral) 
 4. [PDB preparation (from MD)](#preparePDB_MD)
 5. [Interaction Potentials](#interaction)
    1. [Ligand](#ligand)
    2. [Protein](#protein)
 6. [Titration](#titration)
 7. [Questions & Comments](#questions)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***

<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **pdbCode**: PDB code of the protein structure (e.g. 1AKI)

In [1]:
import nglview
import ipywidgets
import plotly
from plotly import subplots
import plotly.graph_objs as go

data_folder = "data/"
pdbCode = "1aki"
inputPDB = data_folder + pdbCode + ".pdb"

MDCode = "RBD-hACE2"
inputPDB_MD = data_folder + MDCode + ".noZN.pdb" 
inputTOP_MD = data_folder + MDCode + ".top" 

<a id="preparePDB"></a>
***
## PDB Preparation (from PDB databank)
CMIP tool needs additional information (e.g. charges, elements) included in the structure PDB file to properly run. In the next cells, a couple of BioBB building blocks will be used to prepare the input PDB file, adding this extra information.   
***
**Building Blocks** used:
 - [prepare_pdb](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.prepare_pdb) from **biobb_cmip.cmip.prepare_pdb**
***

In [2]:
from biobb_cmip.cmip.prepare_pdb import prepare_pdb

cmipPDB = pdbCode + ".cmip.pdb"

prepare_pdb(input_pdb_path=inputPDB,
            output_cmip_pdb_path=cmipPDB)

2022-03-11 12:12:21,154 [MainThread  ] [INFO ]  Not using any container
2022-03-11 12:12:21,154 [MainThread  ] [INFO ]  check_structure -v -i data/1aki.pdb -o 1aki.cmip.pdb --output_format cmip --non_interactive command_list --list 'water --remove yes; backbone --add_caps none; fixside --fix All; add_hydrogen --add_mode auto --add_charges CMIP'

2022-03-11 12:12:21,982 [MainThread  ] [INFO ]  Exit code 0

2022-03-11 12:12:21,984 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.10                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-21                  =

Structure data/1aki.pdb loaded
 Title: the structure of the orthorhombic form of hen egg-white lysozyme at 1.5 angstroms resolution
 Experimental method: x-ray diffraction
 Keywords: hydrolase, glycosidase
 Resolution (A): 1.5

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  207
 Num. r

0

<a id="mips"></a>
***
## Molecular Interaction Potentials (MIPs)
...   
***
**Building Blocks** used:
 - [cmip](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.cmip) from **biobb_cmip.cmip.cmip**
***

In [3]:
from biobb_cmip.cmip.cmip import cmip

mip_pos_log = pdbCode + ".mip_pos.log"
mip_pos_cube = pdbCode + ".mip_pos.cube"

prop = { 
    'execution_type' : 'mip_pos',
    'container_path': 'docker',
    'container_image': 'quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0',
    'container_volume_path': '/inout'
}

cmip(input_pdb_path=cmipPDB,
     output_log_path=mip_pos_log,
     output_cube_path=mip_pos_cube,
     properties=prop)

2022-03-11 12:12:24,741 [MainThread  ] [INFO ]  Copy: 1aki.cmip.pdb to /Users/pau/projects/biobb_cmip/jupyter_notebook/19d5be92-765a-48fd-bc94-c5ffe433c13c
2022-03-11 12:12:24,743 [MainThread  ] [INFO ]  Copy: /Users/pau/anaconda3/envs/biobb/share/cmip/dat/vdwprm to /Users/pau/projects/biobb_cmip/jupyter_notebook/19d5be92-765a-48fd-bc94-c5ffe433c13c
2022-03-11 12:12:24,746 [MainThread  ] [INFO ]  Copy: b9a2764c-8f8c-4db7-bad3-5b9252d4e0f2/params to /Users/pau/projects/biobb_cmip/jupyter_notebook/19d5be92-765a-48fd-bc94-c5ffe433c13c
2022-03-11 12:12:24,748 [MainThread  ] [INFO ]  Using Docker image quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0
2022-03-11 12:12:24,749 [MainThread  ] [INFO ]  docker run -v /Users/pau/projects/biobb_cmip/jupyter_notebook/19d5be92-765a-48fd-bc94-c5ffe433c13c:/inout quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0 /bin/bash -c "cmip -i /inout/params -vdw /inout/vdwprm -hs /inout/1aki.cmip.pdb -cube /inout/1aki.mip_pos.cube -o /inout/1aki.mip_pos.log"

2022-03-11 

0

In [4]:
! cat 1aki.mip_pos.log

 =                                               =
 =                C M I P  (2.7.0)               =
 =          J. Ll. Gelpi, A. Morreale,           =
 =            F. J. Luque, M.Orozco              =
 =      Dept. Biochemistry. Univ. Barcelona      =
 =                  1999-2021                    =

  Code for ASA calculation by Juan F. Recio       

#T Run started at   11:12:26 h on 11- 3-2022
 SIZES
 -----
 MAXTIP:           100
 MAXATH:            80000
 MAXATP x MAXCONF: 100000

 INPUT FILES
 -----------
    Calc. settings: /inout/params                                                                                                                                         
 VdW Parameters:      /inout/vdwprm                                                                                                                                         
 PDB Host:            /inout/1aki.cmip.pdb                                                                                              

In [5]:
view = nglview.show_structure_file(mip_pos_cube)
view.add_component(cmipPDB)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_surface(isolevelType="value", isolevel=-5, color="blue")
view.component_1.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [6]:
from biobb_cmip.cmip.cmip import cmip

mip_neg_log = pdbCode + ".mip_neg.log"
mip_neg_cube = pdbCode + ".mip_neg.cube"

prop = { 
    'execution_type' : 'mip_neg',
    'container_path': 'docker',
    'container_image': 'quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0',
    'container_volume_path': '/inout'
}

cmip(input_pdb_path=cmipPDB,
     output_log_path=mip_neg_log,
     output_cube_path=mip_neg_cube,
     properties=prop)

2022-03-11 12:12:48,959 [MainThread  ] [INFO ]  Copy: 1aki.cmip.pdb to /Users/pau/projects/biobb_cmip/jupyter_notebook/6c1bd0b4-ab01-40eb-9a30-7673eb15e26e
2022-03-11 12:12:48,962 [MainThread  ] [INFO ]  Copy: /Users/pau/anaconda3/envs/biobb/share/cmip/dat/vdwprm to /Users/pau/projects/biobb_cmip/jupyter_notebook/6c1bd0b4-ab01-40eb-9a30-7673eb15e26e
2022-03-11 12:12:48,965 [MainThread  ] [INFO ]  Copy: a39eceb3-812f-4e0c-9f0d-987f399668e2/params to /Users/pau/projects/biobb_cmip/jupyter_notebook/6c1bd0b4-ab01-40eb-9a30-7673eb15e26e
2022-03-11 12:12:48,966 [MainThread  ] [INFO ]  Using Docker image quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0
2022-03-11 12:12:48,967 [MainThread  ] [INFO ]  docker run -v /Users/pau/projects/biobb_cmip/jupyter_notebook/6c1bd0b4-ab01-40eb-9a30-7673eb15e26e:/inout quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0 /bin/bash -c "cmip -i /inout/params -vdw /inout/vdwprm -hs /inout/1aki.cmip.pdb -cube /inout/1aki.mip_neg.cube -o /inout/1aki.mip_neg.log"

2022-03-11 

0

In [7]:
! cat 1aki.mip_neg.log

 =                                               =
 =                C M I P  (2.7.0)               =
 =          J. Ll. Gelpi, A. Morreale,           =
 =            F. J. Luque, M.Orozco              =
 =      Dept. Biochemistry. Univ. Barcelona      =
 =                  1999-2021                    =

  Code for ASA calculation by Juan F. Recio       

#T Run started at   11:12:49 h on 11- 3-2022
 SIZES
 -----
 MAXTIP:           100
 MAXATH:            80000
 MAXATP x MAXCONF: 100000

 INPUT FILES
 -----------
    Calc. settings: /inout/params                                                                                                                                         
 VdW Parameters:      /inout/vdwprm                                                                                                                                         
 PDB Host:            /inout/1aki.cmip.pdb                                                                                              

In [8]:
view = nglview.show_structure_file(mip_neg_cube)
view.add_component(cmipPDB)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_surface(isolevelType="value", isolevel=-5, color="red")
view.component_1.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [9]:
from biobb_cmip.cmip.cmip import cmip

mip_neutral_log = pdbCode + ".mip_neutral.log"
mip_neutral_cube = pdbCode + ".mip_neutral.cube"

prop = { 
    'execution_type' : 'mip_neu',
    'container_path': 'docker',
    'container_image': 'quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0',
    'container_volume_path': '/inout'
}

cmip(input_pdb_path=cmipPDB,
     output_log_path=mip_neutral_log,
     output_cube_path=mip_neutral_cube,
     properties=prop)

2022-03-11 12:13:04,190 [MainThread  ] [INFO ]  Copy: 1aki.cmip.pdb to /Users/pau/projects/biobb_cmip/jupyter_notebook/4f6558f0-f52d-44a6-a945-293ba4730bdf
2022-03-11 12:13:04,193 [MainThread  ] [INFO ]  Copy: /Users/pau/anaconda3/envs/biobb/share/cmip/dat/vdwprm to /Users/pau/projects/biobb_cmip/jupyter_notebook/4f6558f0-f52d-44a6-a945-293ba4730bdf
2022-03-11 12:13:04,195 [MainThread  ] [INFO ]  Copy: 9d458b9d-7b34-4f06-a3ed-7b6b097a6d3e/params to /Users/pau/projects/biobb_cmip/jupyter_notebook/4f6558f0-f52d-44a6-a945-293ba4730bdf
2022-03-11 12:13:04,196 [MainThread  ] [INFO ]  Using Docker image quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0
2022-03-11 12:13:04,196 [MainThread  ] [INFO ]  docker run -v /Users/pau/projects/biobb_cmip/jupyter_notebook/4f6558f0-f52d-44a6-a945-293ba4730bdf:/inout quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0 /bin/bash -c "cmip -i /inout/params -vdw /inout/vdwprm -hs /inout/1aki.cmip.pdb -cube /inout/1aki.mip_neutral.cube -o /inout/1aki.mip_neutral.log"

202

0

In [10]:
view = nglview.show_structure_file(mip_neutral_cube)
view.add_component(cmipPDB)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_surface(isolevelType="value", isolevel=-1, color="grey")
view.component_1.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [11]:
#Show different structures generated (for comparison)
view1 = nglview.show_structure_file(cmipPDB)
view1.add_component(mip_pos_cube)
view1.component_0.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view1.component_1.add_surface(isolevelType="value", isolevel=-5, color="blue")
view1.component_0.center()
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1.camera='orthographic'
view1
view2 = nglview.show_structure_file(cmipPDB)
view2.add_component(mip_neg_cube)
view2.component_0.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view2.component_1.add_surface(isolevelType="value", isolevel=-5, color="red")
view2.component_0.center()
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2.camera='orthographic'
view2
view3 = nglview.show_structure_file(cmipPDB)
view3.add_component(mip_neutral_cube)
view3.component_0.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view3.component_1.add_surface(isolevelType="value", isolevel=-1, color="grey")
view3.component_0.center()
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3.camera='orthographic'
view3
ipywidgets.HBox([view1, view2, view3])

<a id="preparePDB_MD"></a>
***
## PDB Preparation (from Molecular Dynamics)
...
***
**Building Blocks** used:
 - [prepare_structure](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.prepare_structure) from **biobb_cmip.cmip.prepare_structure**
***

In [12]:
from biobb_structure_utils.utils.extract_chain import extract_chain

input_extract_chain = data_folder + "md.imaged.rot.dry.pdb"
inputPDB_MD_hACE2 = MDCode + ".hACE2.pdb"
inputPDB_MD_RBD = MDCode + ".RBD.pdb"

prop = {
    'chains': [ 'A' ],
    'permissive': True
}
extract_chain(input_structure_path=input_extract_chain,
              output_structure_path=inputPDB_MD_hACE2,
              properties=prop)

prop = {
    'chains': [ 'B' ],
    'permissive': True
}
extract_chain(input_structure_path=inputPDB_MD,
              output_structure_path=inputPDB_MD_RBD,
              properties=prop)

2022-03-11 12:13:21,963 [MainThread  ] [INFO ]  Selected Chains: A
2022-03-11 12:13:21,964 [MainThread  ] [INFO ]  Warning: Use permissive=True is a risky option use it under your own responsability
2022-03-11 12:13:21,988 [MainThread  ] [INFO ]  Removed: []
2022-03-11 12:13:21,991 [MainThread  ] [INFO ]  Selected Chains: B
2022-03-11 12:13:21,992 [MainThread  ] [INFO ]  Warning: Use permissive=True is a risky option use it under your own responsability
2022-03-11 12:13:22,017 [MainThread  ] [INFO ]  Removed: []


In [13]:
from biobb_cmip.cmip.prepare_structure import prepare_structure

cmipPDB_MD_hACE2 = MDCode + ".hACE2.cmip.pdb"
cmipPDB_MD_RBD = MDCode + ".RBD.cmip.pdb"

prepare_structure(input_pdb_path=inputPDB_MD_hACE2,
                  input_topology_path=inputTOP_MD,
                  output_cmip_pdb_path=cmipPDB_MD_hACE2)

prepare_structure(input_pdb_path=inputPDB_MD_RBD,
                  input_topology_path=inputTOP_MD,
                  output_cmip_pdb_path=cmipPDB_MD_RBD)

2022-03-11 12:13:22,029 [MainThread  ] [INFO ]  Reading: data/RBD-hACE2.top to extract charges
2022-03-11 12:14:09,723 [MainThread  ] [INFO ]  Reading: data/RBD-hACE2.top to extract elements
2022-03-11 12:15:20,044 [MainThread  ] [INFO ]  Removed: []
2022-03-11 12:15:20,051 [MainThread  ] [INFO ]  Reading: data/RBD-hACE2.top to extract charges
2022-03-11 12:16:07,787 [MainThread  ] [INFO ]  Reading: data/RBD-hACE2.top to extract elements
2022-03-11 12:17:17,864 [MainThread  ] [INFO ]  Removed: []


0

<a id="interaction"></a>
***
## Interaction Potential Energies 
... 
***
**Building Blocks** used:
 - [cmip](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.cmip) from **biobb_cmip.cmip.cmip**
***

In [14]:
from biobb_cmip.cmip.cmip import cmip

######### Remove this #############
cmipPDB_MD_hACE2 = "RBD-hACE2" + ".hACE2.cmip.pdb"
cmipPDB_MD_RBD   = "RBD-hACE2" + ".RBD.cmip.pdb"
data_folder = "data/"
pdbCode = "1aki"
inputPDB = data_folder + pdbCode + ".pdb"

MDCode = "RBD-hACE2"
inputPDB_MD = data_folder + MDCode + ".noZN.pdb" 
inputTOP_MD = data_folder + MDCode + ".top" 
###################################

RBD_energies_log = "RBD.energies.log"
RBD_byat_out = "RBD.energies.byat.out"

prop = { 
    'execution_type' : 'energy',
    'remove_tmp': False
}

cmip(input_pdb_path=cmipPDB_MD_hACE2,
     input_probe_pdb_path=cmipPDB_MD_RBD,
     output_log_path=RBD_energies_log,
     output_byat_path=RBD_byat_out,
     properties=prop)

2022-03-11 12:18:30,718 [MainThread  ] [INFO ]  Not using any container
2022-03-11 12:18:30,720 [MainThread  ] [INFO ]  cmip -i ec17efc7-cac1-4578-9416-32e0b5f681fc/params -vdw /Users/pau/anaconda3/envs/biobb/share/cmip/dat/vdwprm -hs RBD-hACE2.hACE2.cmip.pdb -pr RBD-hACE2.RBD.cmip.pdb -byat RBD.energies.byat.out -o RBD.energies.log

2022-03-11 12:18:51,933 [MainThread  ] [INFO ]  Exit code 0



0

In [15]:
import nglview as nv
from biobb_cmip.utils.representation import create_box_representation

boxedFilename, atomPair = create_box_representation(RBD_energies_log, inputPDB_MD)
# Represent the new file in ngl
view = nv.show_structure_file(boxedFilename, default=False)
# Structure
view.add_representation(repr_type='cartoon', selection='not het', color='#cccccc', opacity=.2)
# ligands box
view.add_representation(repr_type='ball+stick', selection='9999', aspectRatio = 10)
# lines box
view.add_representation(repr_type='distance', atomPair= atomPair, labelColor= 'transparent', color= 'black')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [16]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byat

atom_list, energy_dict = get_energies_byat(RBD_byat_out)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=atom_list, y=energy_dict['ES'])],
       "layout": go.Layout(title="CMIP Interaction Potential", xaxis=dict(title = "Atom Number"), 
                           yaxis=dict(title = "Potential Energy Kcal/mol"))}

plotly.offline.iplot(fig)



In [17]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byres


res_list, energy_dict = get_energies_byres(RBD_byat_out)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=res_list, y=energy_dict['ES'])],
       "layout": go.Layout(title="CMIP Interaction Potential", xaxis=dict(title = "Residue ID"), 
                           yaxis=dict(title = "Potential Energy Kcal/mol"))}

plotly.offline.iplot(fig)



In [20]:
from biobb_cmip.cmip.prepare_structure import prepare_structure


box_pdb_output = 'output_tit_all.pdb'

prepare_structure(input_pdb_path=inputPDB_MD,
                  input_topology_path=inputTOP_MD,
                  output_cmip_pdb_path=box_pdb_output)


2022-03-11 12:19:47,653 [MainThread  ] [INFO ]  Reading: data/RBD-hACE2.top to extract charges
2022-03-11 12:20:36,617 [MainThread  ] [INFO ]  Reading: data/RBD-hACE2.top to extract elements
2022-03-11 12:21:47,666 [MainThread  ] [INFO ]  Removed: []


0

In [46]:
from biobb_cmip.cmip.cmip import cmip

RBD_energies_log = "hACE2.box.log"
RBD_byat_out = "hACE2.box.byat.out"
json_box_out = "box.json"

prop = { 
    'execution_type' : 'check_only',
    'remove_tmp':False,
    'box_size_factor': 0.9
}

cmip(input_pdb_path=box_pdb_output,
     output_log_path=RBD_energies_log,
     output_byat_path=RBD_byat_out,
     output_json_box_path=json_box_out,
     properties=prop)

2022-03-11 13:50:18,771 [MainThread  ] [INFO ]  Not using any container
2022-03-11 13:50:18,772 [MainThread  ] [INFO ]  cmip -i 6e076041-cac7-4f33-81c9-ecc9f11a7a81/params -vdw /Users/pau/anaconda3/envs/biobb/share/cmip/dat/vdwprm -hs output_tit_all.pdb -byat hACE2.box.byat.out -o hACE2.box.log -l 42c98e34-ab9f-49e0-8c57-3aca1cd5c138/key_value_cmip_log.log

2022-03-11 13:50:19,084 [MainThread  ] [INFO ]  Exit code 0

2022-03-11 13:50:19,087 [MainThread  ] [INFO ]  STOP 0



0

In [47]:
! cat hACE2.box.log

 =                                               =
 =                C M I P  (2.7.0)               =
 =          J. Ll. Gelpi, A. Morreale,           =
 =            F. J. Luque, M.Orozco              =
 =      Dept. Biochemistry. Univ. Barcelona      =
 =                  1999-2021                    =

  Code for ASA calculation by Juan F. Recio       

#T Run started at   13:50:18 h on 11- 3-2022
 SIZES
 -----
 MAXTIP:           100
 MAXATH:            80000
 MAXATP x MAXCONF: 100000

 INPUT FILES
 -----------
    Calc. settings: 6e076041-cac7-4f33-81c9-ecc9f11a7a81/params                                                                                                           
 VdW Parameters:      /Users/pau/anaconda3/envs/biobb/share/cmip/dat/vdwprm                                                                                                 
 PDB Host:            output_tit_all.pdb                                                                                                

In [48]:
! cat box.json

{
    "origin": {
        "x": 15.4,
        "y": 11.07,
        "z": 23.9
    },
    "size": {
        "x": 90.0,
        "y": 103.5,
        "z": 73.0
    },
    "params": {
        "CEN": [
            60.4,
            62.82,
            56.315
        ],
        "DIM": [
            180,
            207,
            146
        ],
        "INT": [
            0.5,
            0.5,
            0.5
        ]
    }
}

In [49]:
import nglview as nv
from biobb_cmip.utils.representation import create_box_representation

boxedFilename, atomPair = create_box_representation(json_box_out, inputPDB_MD)
# Represent the new file in ngl
view = nv.show_structure_file(boxedFilename, default=False)
# Structure
view.add_representation(repr_type='cartoon', selection='not het', color='#cccccc', opacity=.2)
# ligands box
view.add_representation(repr_type='ball+stick', selection='9999', aspectRatio = 10)
# lines box
view.add_representation(repr_type='distance', atomPair= atomPair, labelColor= 'transparent', color= 'black')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view



NGLWidget()

In [50]:
!cat output_tit_all.pdb

ATOM      1  N   SER A   1      87.800  28.660  45.150  0.1849  N
ATOM      2  H1  SER A   1      87.790  29.060  44.220  0.1898  HN
ATOM      3  H2  SER A   1      87.680  27.660  45.110  0.1898  HN
ATOM      4  H3  SER A   1      88.700  28.770  45.590  0.1898  HN
ATOM      5  CA  SER A   1      86.720  28.970  46.100  0.0567  C
ATOM      6  HA  SER A   1      85.760  28.680  45.680  0.0782  H
ATOM      7  CB  SER A   1      86.940  28.140  47.360  0.2596  C
ATOM      8  HB2 SER A   1      88.010  28.160  47.570  0.0273  H
ATOM      9  HB3 SER A   1      86.440  28.530  48.250  0.0273  H
ATOM     10  OG  SER A   1      86.560  26.810  47.070 -0.6714  O
ATOM     11  HG  SER A   1      85.900  26.460  47.660  0.4239  HO
ATOM     12  C   SER A   1      86.740  30.450  46.460  0.6163  C
ATOM     13  O   SER A   1      87.800  31.070  46.400 -0.5722  O
ATOM     14  N   THR A   2      85.600  30.960  46.920 -0.4157  N
ATOM     15  H   THR A   2      84.880  30.260  47.070  0.2719  HN
ATOM 

ATOM   2897  O   GLY A 182      86.390  65.560  56.570 -0.5679  O
ATOM   2898  N   ASP A 183      88.490  64.760  56.660 -0.5163  N
ATOM   2899  H   ASP A 183      89.360  64.790  57.170  0.2936  HN
ATOM   2900  CA  ASP A 183      88.300  63.750  55.640  0.0381  C
ATOM   2901  HA  ASP A 183      88.060  64.270  54.710  0.0880  H
ATOM   2902  CB  ASP A 183      89.500  62.830  55.470 -0.0303  C
ATOM   2903  HB2 ASP A 183      89.990  62.510  56.390 -0.0122  H
ATOM   2904  HB3 ASP A 183      89.150  61.840  55.180 -0.0122  H
ATOM   2905  CG  ASP A 183      90.500  63.330  54.440  0.7994  C
ATOM   2906  OD1 ASP A 183      90.040  64.140  53.600 -0.8014  O
ATOM   2907  OD2 ASP A 183      91.640  62.870  54.210 -0.8014  O
ATOM   2908  C   ASP A 183      87.100  62.820  55.810  0.5366  C
ATOM   2909  O   ASP A 183      86.610  62.370  54.780 -0.5819  O
ATOM   2910  N   TYR A 184      86.570  62.580  57.010 -0.4157  N
ATOM   2911  H   TYR A 184      87.080  63.040  57.750  0.2719  HN
ATOM   2

ATOM   6658  NZ  LYS A 423      62.320  89.290  52.860 -0.3854  N
ATOM   6659  HZ1 LYS A 423      62.080  88.930  53.770  0.3400  HN
ATOM   6660  HZ2 LYS A 423      62.340  90.290  52.980  0.3400  HN
ATOM   6661  HZ3 LYS A 423      61.610  89.110  52.170  0.3400  HN
ATOM   6662  C   LYS A 423      67.450  88.250  48.820  0.7341  C
ATOM   6663  O   LYS A 423      67.810  87.600  49.800 -0.5894  O
ATOM   6664  N   GLN A 424      67.380  87.690  47.610 -0.4157  N
ATOM   6665  H   GLN A 424      66.960  88.180  46.840  0.2719  HN
ATOM   6666  CA  GLN A 424      67.810  86.350  47.260 -0.0031  C
ATOM   6667  HA  GLN A 424      67.360  85.740  48.050  0.0850  H
ATOM   6668  CB  GLN A 424      67.380  85.830  45.900 -0.0036  C
ATOM   6669  HB2 GLN A 424      67.680  86.470  45.070  0.0171  H
ATOM   6670  HB3 GLN A 424      67.760  84.810  45.890  0.0171  H
ATOM   6671  CG  GLN A 424      65.850  85.750  45.860 -0.0645  C
ATOM   6672  HG2 GLN A 424      65.510  85.230  46.750  0.0352  H
ATOM  


ATOM  11940  HB2 TRP B 755      68.540  33.120  62.140  0.1123  H
ATOM  11941  HB3 TRP B 755      67.650  31.680  62.300 -0.0050  C
ATOM  11942  CG  TRP B 755      69.190  31.700  63.650  0.0339  H
ATOM  11943  CD1 TRP B 755      70.050  32.390  64.440  0.0339  H
ATOM  11944  HD1 TRP B 755      70.470  33.340  64.140 -0.1415  C
ATOM  11945  NE1 TRP B 755      70.360  31.670  65.570 -0.1638  C
ATOM  11946  HE1 TRP B 755      71.060  31.870  66.270  0.2062  H
ATOM  11947  CE2 TRP B 755      69.700  30.450  65.600 -0.3418  N
ATOM  11948  CZ2 TRP B 755      69.670  29.380  66.490  0.3412  HN
ATOM  11949  HZ2 TRP B 755      70.140  29.430  67.460  0.1380  C
ATOM  11950  CH2 TRP B 755      68.980  28.220  66.090 -0.2601  C
ATOM  11951  HH2 TRP B 755      69.000  27.350  66.730  0.1572  H
ATOM  11952  CZ3 TRP B 755      68.260  28.210  64.890 -0.1134  C
ATOM  11953  HZ3 TRP B 755      67.710  27.310  64.700  0.1417  H
ATOM  11954  CE3 TRP B 755      68.240  29.330  64.040 -0.1972  C
ATOM  11

In [51]:
%load_ext autoreload
%autoreload 2

from biobb_cmip.cmip.ignore_residues import ignore_residues


inputPDB_MD_hACE2_ignored = "inputPDB_MD_hACE2_ignored.pdb"




prop = {
    'residue_list': "A:"
}

ignore_residues(input_cmip_pdb_path = box_pdb_output,
               output_cmip_pdb_path = inputPDB_MD_hACE2_ignored,
               properties = prop)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2022-03-11 13:50:59,832 [MainThread  ] [INFO ]  Residue list: ['A:']
2022-03-11 13:50:59,916 [MainThread  ] [INFO ]  1 residues have been marked
2022-03-11 13:50:59,918 [MainThread  ] [INFO ]  Removed: []


In [52]:
! cat inputPDB_MD_hACE2_ignored.pdb

ATOM      1  N   SER A   1      87.800  28.660  45.150  0.1849  XN
ATOM      2  H1  SER A   1      87.790  29.060  44.220  0.1898  XHN
ATOM      3  H2  SER A   1      87.680  27.660  45.110  0.1898  XHN
ATOM      4  H3  SER A   1      88.700  28.770  45.590  0.1898  XHN
ATOM      5  CA  SER A   1      86.720  28.970  46.100  0.0567  XC
ATOM      6  HA  SER A   1      85.760  28.680  45.680  0.0782  XH
ATOM      7  CB  SER A   1      86.940  28.140  47.360  0.2596  XC
ATOM      8  HB2 SER A   1      88.010  28.160  47.570  0.0273  XH
ATOM      9  HB3 SER A   1      86.440  28.530  48.250  0.0273  XH
ATOM     10  OG  SER A   1      86.560  26.810  47.070 -0.6714  XO
ATOM     11  HG  SER A   1      85.900  26.460  47.660  0.4239  XHO
ATOM     12  C   SER A   1      86.740  30.450  46.460  0.6163  XC
ATOM     13  O   SER A   1      87.800  31.070  46.400 -0.5722  XO
ATOM     14  N   THR A   2      85.600  30.960  46.920 -0.4157  XN

ATOM     16  CA  THR A   2      85.360  32.380  47.060 -0

ATOM   3707  ND2 ASN A 232      73.470 107.100  53.440 -0.9191  XN
ATOM   3708 1HD2 ASN A 232      74.110 107.470  52.750  0.4196  XHN
ATOM   3709 2HD2 ASN A 232      72.460 107.080  53.410  0.4196  XHN
ATOM   3710  C   ASN A 232      72.610 104.960  57.870  0.5973  XC
ATOM   3711  O   ASN A 232      72.120 105.810  58.610 -0.5679  XO
ATOM   3712  N   ALA A 233      72.130 103.720  57.840 -0.4157  XN
ATOM   3713  H   ALA A 233      72.430 103.040  57.150  0.2719  XHN
ATOM   3714  CA  ALA A 233      71.210 103.300  58.880  0.0337  XC
ATOM   3715  HA  ALA A 233      70.350 103.960  58.770  0.0823  XH
ATOM   3716  CB  ALA A 233      70.770 101.880  58.530 -0.1825  XC
ATOM   3717  HB1 ALA A 233      71.540 101.150  58.740  0.0603  XH
ATOM   3718  HB2 ALA A 233      69.980 101.580  59.220  0.0603  XH
ATOM   3719  HB3 ALA A 233      70.390 101.740  57.520  0.0603  XH
ATOM   3720  C   ALA A 233      71.700 103.440  60.310  0.5973  XC
ATOM   3721  O   ALA A 233      70.910 103.750  61.200 -0.5

ATOM   8244  HE2 HIE A 517      62.700  82.010  33.410  0.3339  XHN
ATOM   8245  CD2 HIE A 517      62.500  80.130  32.410 -0.2207  XC
ATOM   8246  HD2 HIE A 517      63.500  79.750  32.240  0.1862  XH
ATOM   8247  C   HIE A 517      61.770  78.780  29.090  0.5973  XC
ATOM   8248  O   HIE A 517      62.870  78.250  28.960 -0.5679  XO
ATOM   8249  N   GLU A 518      61.500  79.960  28.520 -0.5163  XN
ATOM   8250  H   GLU A 518      60.650  80.400  28.850  0.2936  XHN
ATOM   8251  CA  GLU A 518      62.410  80.780  27.740  0.0397  XC
ATOM   8252  HA  GLU A 518      63.200  80.110  27.400  0.1105  XH
ATOM   8253  CB  GLU A 518      61.570  81.270  26.570  0.0560  XC
ATOM   8254  HB2 GLU A 518      61.170  80.360  26.120 -0.0173  XH
ATOM   8255  HB3 GLU A 518      60.740  81.820  27.000 -0.0173  XH
ATOM   8256  CG  GLU A 518      62.220  82.160  25.510  0.0136  XC
ATOM   8257  HG2 GLU A 518      61.450  82.660  24.940 -0.0425  XH
ATOM   8258  HG3 GLU A 518      62.870  82.910  25.960 -0.04

In [53]:
from biobb_cmip.cmip.cmip import cmip

RBD_energies_log = "hACE2.energies.log"
RBD_byat_out = "hACE2.energies.byat.out"

prop = { 
    'execution_type' : 'energy',
    'remove_tmp':False,
}

cmip(input_pdb_path=inputPDB_MD_hACE2_ignored,
     input_probe_pdb_path=cmipPDB_MD_hACE2,
     output_log_path=RBD_energies_log,
     output_byat_path=RBD_byat_out,
     input_json_box_path=json_box_out,
     properties=prop)

2022-03-11 13:51:42,505 [MainThread  ] [INFO ]  Not using any container
2022-03-11 13:51:42,507 [MainThread  ] [INFO ]  cmip -i 6c0549bb-8c88-4c90-acb2-8a91907da332/params -vdw /Users/pau/anaconda3/envs/biobb/share/cmip/dat/vdwprm -hs inputPDB_MD_hACE2_ignored.pdb -pr RBD-hACE2.hACE2.cmip.pdb -byat hACE2.energies.byat.out -o hACE2.energies.log

2022-03-11 13:53:40,277 [MainThread  ] [INFO ]  Exit code 0



0

In [54]:
! cat hACE2.energies.log

 =                                               =
 =                C M I P  (2.7.0)               =
 =          J. Ll. Gelpi, A. Morreale,           =
 =            F. J. Luque, M.Orozco              =
 =      Dept. Biochemistry. Univ. Barcelona      =
 =                  1999-2021                    =

  Code for ASA calculation by Juan F. Recio       

#T Run started at   13:51:42 h on 11- 3-2022
 SIZES
 -----
 MAXTIP:           100
 MAXATH:            80000
 MAXATP x MAXCONF: 100000

 INPUT FILES
 -----------
    Calc. settings: 6c0549bb-8c88-4c90-acb2-8a91907da332/params                                                                                                           
 VdW Parameters:      /Users/pau/anaconda3/envs/biobb/share/cmip/dat/vdwprm                                                                                                 
 PDB Host:            inputPDB_MD_hACE2_ignored.pdb                                                                                     

In [55]:
import nglview as nv
from biobb_cmip.utils.representation import create_box_representation

boxedFilename, atomPair = create_box_representation(RBD_energies_log, inputPDB_MD)
# Represent the new file in ngl
view = nv.show_structure_file(boxedFilename, default=False)
# Structure
view.add_representation(repr_type='cartoon', selection='not het', color='#cccccc', opacity=.2)
# ligands box
view.add_representation(repr_type='ball+stick', selection='9999', aspectRatio = 10)
# lines box
view.add_representation(repr_type='distance', atomPair= atomPair, labelColor= 'transparent', color= 'black')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [56]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byat


atom_list, energy_dict = get_energies_byat(RBD_byat_out)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=atom_list, y=energy_dict['ES'])],
       "layout": go.Layout(title="CMIP Interaction Potential", xaxis=dict(title = "Atom Number"), 
                           yaxis=dict(title = "Potential Energy Kcal/mol"))}

plotly.offline.iplot(fig)



In [57]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byres


res_list, energy_dict = get_energies_byres(RBD_byat_out)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=res_list, y=energy_dict['ES'])],
       "layout": go.Layout(title="CMIP Interaction Potential", xaxis=dict(title = "Residue ID"), 
                           yaxis=dict(title = "Interaction Energy Kcal/mol"))}

plotly.offline.iplot(fig)



<a id="titration"></a>
***
## PDB Titration
...
***
**Building Blocks** used:
 - [titration](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.titration) from **biobb_cmip.cmip.titration**
 - [cat_pdb](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.cat_pdb) from **biobb_structure_utils.utils.cat_pdb**
***

In [ ]:
from biobb_cmip.cmip.titration import titration

titration_input_pdb = data_folder + 'input2.pdb'
titration_output_pdb = 'output_tit.pdb'
titration_output_log = 'output_tit.log' 

prop = { 
    'num_positive_ions' : 5,
    'num_negative_ions' : 5,    
    'num_wats' : 20,
    'container_path': 'docker',
    'container_image': 'quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0',
    'container_volume_path': '/inout'
}

titration(input_pdb_path=titration_input_pdb,
          output_pdb_path=titration_output_pdb,
          output_log_path=titration_output_log,
          properties=prop)

In [ ]:
from biobb_structure_utils.utils.cat_pdb import cat_pdb

cat_pdb_output = 'output_tit_all.pdb'

cat_pdb(input_structure1=titration_input_pdb,
        input_structure2=titration_output_pdb,
        output_structure_path=cat_pdb_output)

In [ ]:
view = nglview.show_structure_file(cat_pdb_output)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='spacefill', selection='water')
view.add_representation(repr_type='spacefill', selection='.Na', color='element')
view.add_representation(repr_type='spacefill', selection='.Cl', color='element')
view._remote_call('setSize', target='Widget', args=['','600px'])
view